In [1]:
import pandas as pd

In [40]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [59]:
df = pd.read_json('reviews_Office_Products_5.json', lines=True)
class TextPipeline:

    def __init__(self, df, column, stop_words):
        self.df = df
        self.column = column
        self.stop_words = stop_words

    def _to_lower(self):
        df[self.column] = df[self.column].apply(lambda x: str(x).lower())
        return df

    def _remove_punctuations(self):
        
        def udf_remove_punctuation(txt):
            for punctuaution in string.punctuation:
                txt = txt.replace(punctuaution, '')
            return txt
        
        df[self.column] = df[self.column].apply(lambda x : udf_remove_punctuation(str(x)))

        return df

    def _remove_stopwords(self, tokenized_text, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):

        filtered_sentence = [w for w in tokenized_text if not w in stop_words]

        return filtered_sentence

        
    def _tokenization(self):

        tokenized_text = []
        text = df[self.column].tolist()
        for sentence in text:
            # temp_text = word_tokenize(text)
            temp_text = re.sub('\S*@\S*\s?', '', sentence) 
            temp_text = re.sub('\s+', ' ', sentence)
            temp_text = re.sub("\'", "", sentence)
            temp_text = gensim.utils.simple_preprocess(str(sentence), deacc=True)
            temp_text =  self._remove_stopwords(temp_text,stop_words=stop_words)
            tokenized_text.append(temp_text)
            
        return tokenized_text


    def _models(self, text_native):
        bigram = gensim.models.Phrases(text_native, min_count=5, threshold=100) # higher threshold fewer phrases.
        trigram = gensim.models.Phrases(bigram[text_native], threshold=100)  
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        trigram_mod = gensim.models.phrases.Phraser(trigram)

        return bigram_mod, trigram_mod

    def _process_models(self, texts,  bigram_mod, trigram_mod, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'] ):
        texts = [bigram_mod[doc] for doc in texts]
        texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
        texts_out = []
        nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        # remove stopwords once more after lemmatization
        texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
        return texts_out

    def fit(self):
        prep = self._to_lower()
        prep = self._remove_punctuations()
        tokenized_text = self._tokenization()
        bigram_mod, trigram_mod = self._models(tokenized_text)
        text_out = self._process_models( tokenized_text,  bigram_mod, trigram_mod, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'] )
        return text_out



In [63]:
import string
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
pip = TextPipeline(df,"reviewText", stop_words)
x = pip.fit()
x

[nltk_data] Downloading package punkt to C:\Users\Shantam
[nltk_data]     Saxena\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['buy',
  'first',
  'serve',
  'faithfully',
  'lose',
  'travelling',
  'search',
  'replace',
  'find',
  'difficult',
  'come',
  'area',
  'decide',
  'buy',
  'purchase',
  'mistake',
  'know',
  'many',
  'people',
  'view',
  'flagship',
  'line',
  'disasterthe',
  'powerful',
  'use',
  'extremely',
  'manual',
  'sketchy',
  'good',
  'hand',
  'need',
  'way',
  'make',
  'good',
  'sense',
  'meif',
  'look',
  'solid',
  'reliable',
  'calculator',
  'programmable',
  'basic',
  'statistic',
  'well',
  'many',
  'business',
  'application',
  'manual',
  'make',
  'sense',
  'run',
  'short',
  'orderim',
  'ready',
  'set',
  'aside',
  'move',
  'back',
  'old',
  'friend',
  'nt',
  'even',
  'replace',
  'battery',
  'well',
  'decade',
  'usehp',
  'come',
  'home'],
 ['belate',
  'review',
  'feel',
  'obliged',
  'share',
  'view',
  'old',
  'workhorse',
  'gold',
  'th_anniversary',
  'oth',
  'year',
  'satisfy',
  'ower',
  'user',
  'fan',
  'especially',
  

In [64]:
data = x
data

[['buy',
  'first',
  'serve',
  'faithfully',
  'lose',
  'travelling',
  'search',
  'replace',
  'find',
  'difficult',
  'come',
  'area',
  'decide',
  'buy',
  'purchase',
  'mistake',
  'know',
  'many',
  'people',
  'view',
  'flagship',
  'line',
  'disasterthe',
  'powerful',
  'use',
  'extremely',
  'manual',
  'sketchy',
  'good',
  'hand',
  'need',
  'way',
  'make',
  'good',
  'sense',
  'meif',
  'look',
  'solid',
  'reliable',
  'calculator',
  'programmable',
  'basic',
  'statistic',
  'well',
  'many',
  'business',
  'application',
  'manual',
  'make',
  'sense',
  'run',
  'short',
  'orderim',
  'ready',
  'set',
  'aside',
  'move',
  'back',
  'old',
  'friend',
  'nt',
  'even',
  'replace',
  'battery',
  'well',
  'decade',
  'usehp',
  'come',
  'home'],
 ['belate',
  'review',
  'feel',
  'obliged',
  'share',
  'view',
  'old',
  'workhorse',
  'gold',
  'th_anniversary',
  'oth',
  'year',
  'satisfy',
  'ower',
  'user',
  'fan',
  'especially',
  

In [65]:
# Create Dictionary
id2word = corpora.Dictionary(data)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [ ]:
lda_model.save('lda.model')

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis